
Install the libraries

```
!pip install --upgrade torch transformers accelerate
```

In [1]:
!pip install transformers==4.40.1
!pip install torch==2.2.2
!pip install accelerate==0.29.2
!pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 160.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 149.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 177.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 125.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 150.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 190.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 214.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 90.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 218.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 207.2 MB/s eta 0:00:000

In [2]:
import os
import shutil
from pathlib import Path

import torch
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
from utils import recreate_folder

In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
project_name = os.environ['DOMINO_PROJECT_NAME']
save_path = f"/home/ubuntu/TinyLlama"  # or any suitable path
recreate_folder(save_path)


In [4]:
# Load the model and tokenizer from Hugging Face
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Save them locally
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved locally at {save_path}")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Model saved locally at /home/ubuntu/TinyLlama


## Test the downloaded model locally

In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

def load_local_llm(model_path):
    """Load a locally stored LLM model from disk."""
    #tokenizer = AutoTokenizer.from_pretrained(model_path)
    #model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")

    print(f"{model_name} loaded in float16 on CPU")
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_text(llm_pipeline, prompt, max_length=50):
    """Generate text using the loaded LLM."""
    response = llm_pipeline(prompt, max_length=max_length, do_sample=True, temperature=0.7)
    return response[0]['generated_text']

In [8]:
model_path = save_path  # Change this to your local model directory
model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text-generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time in a distant land,"
output = text_generator(prompt, max_length=50, do_sample=True)
print(output[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Once upon a time in a distant land, there was a kingdom named Nimruz. This kingdom prospered because it had a wise and honorable king, who was a great patron of the arts, especially music. The king heard of a
